In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [6]:
dfs = []

draft_years = list(np.arange(2011, 2022))

for year in draft_years:
    url = "https://www.basketball-reference.com/draft/NBA_" + str(year) + ".html"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    table_body = soup.find("tbody")
    rows = table_body.find_all("tr", class_=False)

    def parse_html(row, data_stat):
        return row.find("td", attrs={"data-stat": data_stat}).text

    data = []
    for row in rows:
        # get name, pick, college team, ws/48, bpm, vorp
        name = row.select_one('a[href^="/players/"]').text
        pick = row.find('th', class_="right", attrs={"data-stat": "ranker"}).text
        college = parse_html(row, "college_name")
        seasons = parse_html(row, "seasons")
        g = parse_html(row, "g")
        pts = parse_html(row, "pts")
        trb = parse_html(row, "trb")
        ast = parse_html(row, "ast")
        ws_per_48 = parse_html(row, "ws_per_48")
        bpm = parse_html(row, "bpm")
        vorp = parse_html(row, "vorp")

        data.append([year, name, pick, college, seasons, g, pts, trb, ast, ws_per_48, bpm, vorp])
    
    df = pd.DataFrame(data, columns=["year", "name", "pick", "college", "seasons", "g", "pts", "trb", "ast", "ws_48", "bpm", "vorp"])
    dfs.append(df)

In [7]:
df = pd.concat(dfs)
df.to_csv("draft.csv", index=False)